# Run PEST and Parallel PEST
### This should be super easy ~

In [ ]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil
import sys
import pyemu
import flopy
import swatmf
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatmf.__file__

In [ ]:
from swatmf import swatmf_pst_par, swatmf_pst_utils, analyzer

# 01. Specify a path to the model folder

In [ ]:
prj_dir = "D:/test"
os.chdir(prj_dir)

In [ ]:
main_opt_path = os.path.join(prj_dir, 'main_opt')

# 02. Initial run with PEST settings for adjusting weights

In [ ]:
pst_name = 'mb_zon.pst'
pyemu.os_utils.run(f"pestpp-glm {pst_name}",cwd=main_opt_path)

## 02.01 Change weights to make all of observation data visible 

In [ ]:
# reweight
pst = pyemu.Pst(os.path.join(main_opt_path, pst_name))
pst.phi

In [ ]:
balanced_groups = {grp:1000 for grp in pst.nnz_obs_groups}
# print(pst.phi) #assumes "my.res" is colocated with "my.pst"
pst.adjust_weights(obsgrp_dict=balanced_groups)
pst.control_data.noptmax = 30

In [ ]:
pst_name_rw = 'mb_zon_rw.pst'

In [ ]:
pst.write(os.path.join(main_opt_path, pst_name_rw), version=2)

# 03. Let's perform a history matching in parallel processing

In [ ]:
num_workers = psutil.cpu_count(logical=False)
num_workers

In [ ]:
os.getcwd()

In [ ]:
m_d = os.path.join(prj_dir, "mb_zon_rw_glm")

In [ ]:
pyemu.os_utils.start_workers(main_opt_path,"pestpp-glm",'mb_zon_rw.pst',
                            num_workers=num_workers,
                            worker_root='.',
                            master_dir=m_d)

In [ ]:
os.chdir(wd)

# 04. Let's see how we did:

In [ ]:
analyzer.phi_progress_plot(pst_name_rw)

In [ ]:
pst = pyemu.Pst(os.path.join(m_d, pst_name_rw))
pst.phi

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
figs = pst.plot(kind="1to1")